In [1]:
import os 
import importlib
import textwrap
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, get_response_synthesizer, PromptHelper
from llama_index.text_splitter import SentenceSplitter
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor
from llama_index.node_parser import SimpleNodeParser
from llama_index.vector_stores import ChromaVectorStore
from llamaindex_object_array_reader.dataset import simple_ols # import a simple dataset 
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from llama_index.indices.query.schema import QueryBundle
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import BitsAndBytesConfig
from llama_index.llms import Ollama
from llama_index import ServiceContext, set_global_tokenizer
# from langchain.embeddings import HuggingFaceEmbedding, HuggingFaceInstructEmbeddings
from llama_index.embeddings import HuggingFaceEmbedding
from transformers import AutoTokenizer, AutoModel
from argparse import Namespace
from chromadb import Collection, PersistentClient
from dotenv import load_dotenv
from llamaindex_object_array_reader import ObjectArrayReader


/Users/yuwang/Developments/python/llamaindex_object_array_reader/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import logging
import sys
from llamaindex_object_array_reader._logging import logger

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
log = logger

In [3]:
# Obsolete
# if os.path.exists('my_cred.py'):
#     my_cred = importlib.import_module('my_cred')
#     os.environ['OPENAI_API_KEY'] = my_cred.OPENAI_API_KEY
# else:
#     # Set your OPENAI API Key
#     os.environ['OPENAI_API_KEY'] = "vy-...cH5N"

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
HF_TOKEN = os.environ['HF_TOKEN']

In [4]:
def print_resp(msg, max_len:int=55):
    """将文本分割为每行最大长度的子字符串
    """
    divider: str = '\n'+ '*'*60+'\n'
    msg = textwrap.fill(msg, width=max_len)
    print(f"""\u2705 RESPONSE:{divider}\n{msg}\n{divider} \U0001F6A9END OF RESPONSE""")

In [5]:
models:Namespace = Namespace(
    BERT_BASE_CHINESE="bert-base-chinese",
    LLAMA2_CHINESE_7B_CHAT="FlagAlpha/Llama2-Chinese-7b-Chat", #18G needed
    LLAMA2_7B_CHAT_HF="meta-llama/Llama-2-7b-chat-hf", #18G needed
    BLOOM_560M="bigscience/bloom-560m", #18G needed
    BLOOMZ_560M="bigscience/bloomz-560m", #18G needed
    GPT2="GPT2", #18G needed
    ALL_MPNET_BASE_V2="sentence-transformers/all-mpnet-base-v2", #18G needed
    MISTRAL_7B_INSTRUCT_V0_1="mistralai/Mistral-7B-Instruct-v0.1", #18G needed
    STARLING_LM_7B="berkeley-nest/Starling-LM-7B-alpha",
)

In [6]:
# Set the check point
check_point:str = models.ALL_MPNET_BASE_V2

In [7]:
tokenizer = AutoTokenizer.from_pretrained(check_point)
set_global_tokenizer(tokenizer)

# Alternatively, using a local LLM
USE_LOCAL:bool = True
if USE_LOCAL:
    # llm = Ollama(model="llama2-chinese")
    llm = Ollama(model="starling-lm:7b-alpha-q3_K_M")
    
else: 
    llm = HuggingFaceLLM(
        model_name=check_point,
        tokenizer_name=check_point,
        context_window=512,
        model_kwargs={
            # 'torch_dtype':torch.float16,
            "token": HF_TOKEN,
            'load_in_8bit':False, #No, the bitsandbytes library only works on CUDA GPU. So it must set to 'False' as running on mac os. 
            'offload_folder':"offload_folder",
            'offload_state_dict':True,
            'is_decoder': True if check_point==models.BERT_BASE_CHINESE else None,
            },
        tokenizer_kwargs={
            "token": HF_TOKEN,
            "return_tensors":'pt',},
        device_map="auto" if check_point!=models.BERT_BASE_CHINESE else "mps", 
    )


In [8]:
embedding_model = HuggingFaceEmbedding(
    model_name=check_point,
    tokenizer=tokenizer,
    cache_folder="cache_folder",
    max_length=512,
    device="mps"
)

In [9]:
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=64)
prompt_helper = PromptHelper(
    context_window=512,
    num_output=256,
    chunk_overlap_ratio=0.1,
    chunk_size_limit=None,
)

In [10]:
documents = SimpleDirectoryReader("test_docs/simple_txt_short_en").load_data()

In [11]:
# Assuming documents have already been loaded
# Initialize the parser
parser = SimpleNodeParser.from_defaults(chunk_size=512, chunk_overlap=20)
# Parse documents into nodes
nodes = parser.get_nodes_from_documents(documents)
print('Total nodes:', len(nodes))
for _, n in enumerate(nodes):
    print(n)
    print('---')

Total nodes: 3
Node ID: ec94fbc2-06cf-41aa-b156-9c8753d101d1
Text: You can do data integration, management, analysis and composing
reports and dashboards with Pharmquer, and then automatize all your
works.
---
Node ID: 91c1aaef-6216-4242-b187-906db3939929
Text: Colosscious' flagship product, Pharmquer, is an enterprise level
software of manufacturing and business intelligence, which is
architected especially for the industry.
---
Node ID: 1dc13ad7-d352-4219-a63c-f174adb3c933
Text: Welcome to Colosscious.  We are the expert who spotlight-focus
on providing the digital technology to bio and pharmaceutical
companies, engaging in boosting the performances of new drug
developments, quality control, manufacturing processes, and reducing
the costs and duration by Big Data.
---


In [12]:
V_DB_NAME = "chromadb"
chroma_client = PersistentClient(V_DB_NAME)
COLLECTION_NAME:str = 'test'
chroma_collection:Collection = chroma_client.get_or_create_collection(COLLECTION_NAME)
vector_store = ChromaVectorStore(chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


2024-02-08 15:18:35,622 - chromadb.telemetry.product.posthog - INFO - (posthog.py:20) - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information. 


Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [13]:
for n in nodes:
    print(storage_context.docstore.document_exists(n.id_))

False
False
False


## Create and store new embeddings to ChromaDB. 

In [ ]:
storage_context.docstore.add_documents(nodes)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model, text_splitter=text_splitter,
    prompt_helper=prompt_helper,)
# index = VectorStoreIndex.from_documents(
#     documents, service_context=service_context, storage_context=storage_context, show_progress=True,
# )
index = VectorStoreIndex(
    nodes, service_context=service_context, storage_context=storage_context, show_progress=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Generating embeddings: 100%|██████████| 3/3 [00:00<00:00,  5.29it/s]
2024-02-07 22:47:49,898 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35 


Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35


2024-02-07 22:47:49,898 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862 


Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862


2024-02-07 22:47:49,899 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: cb553733-838a-421b-89bf-c582fe90182a 


Add of existing embedding ID: cb553733-838a-421b-89bf-c582fe90182a


In [ ]:
# example: 
# "GPT4 Correct User: {prompt}<|end_of_turn|>GPT4 Correct Assistant: {response}<|end_of_turn|>GPT4 Correct User: {follow_up_question}<|end_of_turn|>GPT4 Correct Assistant:"
# ref: https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha
sep = '<|end_of_turn|>'
resp_prompt_temp = "GPT4 Correct Assistant: "

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
tokenizer(
    ["What Colosscious do?"],
    return_tensors="pt",
    add_special_tokens=False,
).input_ids.to("mps")

tensor([[ 2058,  8906, 15098, 18440,  2083,  1033]], device='mps:0')

In [ ]:
query_resp = query_engine.query("What is flagship product of Colosscious")

print_resp(query_resp.response)

2024-02-07 19:52:25,347 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

Sorry, I cannot answer your query without using any
more tools.

************************************************************
 🚩END OF RESPONSE


In [ ]:
query_engine = index.as_chat_engine()
query_resp = query_engine.query("What is Pharmquer?")
print_resp(query_resp.response)

2024-02-07 19:52:36,318 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

PharmQuer is an international pharmacovigilance
electronic system used in more than 80 countries for
the collection and analysis of spontaneous case reports
(adverse reactions to drugs). It is a free, web-based
platform that allows users to report, review and
analyze cases. The primary purpose of PharmQuer is to
facilitate data sharing between regulatory agencies,
pharmaceutical companies, academia, and other
stakeholders in the field of pharmacovigilance.

************************************************************
 🚩END OF RESPONSE


## Load existing embeddings in ChromaDB.

In [23]:
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model, text_splitter=text_splitter,
    prompt_helper=prompt_helper,)
# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context, service_context=service_context
)


In [24]:
# create a query engine
query_engine = index.as_query_engine()

In [25]:
response = query_engine.query("What is Colosscious?")
print_resp(response.response)

2024-02-08 13:58:07,575 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35 


Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35


2024-02-08 13:58:07,576 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862 


Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862


2024-02-08 13:58:07,577 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: cb553733-838a-421b-89bf-c582fe90182a 


...

HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


2024-02-08 13:58:52,140 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


2024-02-08 13:58:57,621 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

 Colosscious, also known as "Colosconscious," is a
specialized entity dedicated to providing digital
technology solutions for bio and pharmaceutical
companies. Their primary objectives include enhancing
new drug development effectiveness, upholding quality
standards, optimizing manufacturing processes, and
reducing expenses related to these domains. It's
essential to acknowledge that the context provided may
contain a typo (Colosconscious instead of Colosscious),
which could result in confusion. While it's possible
that the accurate name is "Colosconscious," without
additional clarification or detailed context, this
assumption cannot be confirmed with certainty.

************************************************************
 🚩END OF RESPONSE


## Use llama_index_object_array_reader

In [14]:
# Preview: demo data
simple_ols[:2]

[{'x1': 97.98219999874924,
  'x2': 99.84941752810117,
  'x3': 100.9727776594234,
  'y': 360.87650920565545},
 {'x1': 101.00077953260389,
  'x2': 99.87874921228179,
  'x3': 99.35642250227457,
  'y': 361.50488035486944}]

In [15]:
loader = ObjectArrayReader()

In [16]:
from llama_index.readers.schema.base import Document
object_arrays:list[Document] = loader.load_data(file=simple_ols)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      50 non-null     float64
 1   x2      50 non-null     float64
 2   x3      50 non-null     float64
 3   y       50 non-null     float64
dtypes: float64(4)
memory usage: 1.7 KB


In [17]:
import pandas as pd
df = pd.DataFrame(simple_ols)

In [18]:
object_arrays[0]

Document(id_='1f219b5d-0518-4048-a8fa-c08a8e0ec816', embedding=None, metadata={'columns': "['x1', 'x2', 'x3', 'y']", 'schema': 'None', 'shape': '(50, 4)'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='01044d00146a997fca8953cf8ba579cb4492a76d451ca3aa31645e0e2e9bcb89', text='97.98219999874924, 99.84941752810117, 100.9727776594234, 360.87650920565545\n101.00077953260389, 99.87874921228179, 99.35642250227457, 361.50488035486944\n98.5109626677227, 100.7485502397903, 99.46465098250788, 359.8117609861218\n100.77335929310553, 100.03722922045552, 99.86657209922947, 362.2336960397953\n100.97359840386007, 99.1724799721807, 100.16093297144785, 362.1391160315852\n100.18799255929102, 100.55900119891184, 100.61532849440285, 363.29752977180965\n100.9157547652626, 98.61649241995889, 99.06726035297895, 359.7975894964005\n101.04615952660859, 102.00920930524853, 100.16419028246959, 364.8003752715575\n99.46321248760913, 100.23898461781165, 100.4603474082993, 361.9

In [22]:
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model,)
index = VectorStoreIndex.from_documents(
    documents=object_arrays, service_context=service_context,  storage_context=storage_context, show_progress=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Generating embeddings: 100%|██████████| 4/4 [00:00<00:00, 10.85it/s]
2024-02-08 15:20:20,129 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35 


Add of existing embedding ID: dc0f865e-90c8-42b0-9239-19625ebcef35


2024-02-08 15:20:20,130 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862 


Add of existing embedding ID: 1f7abdb8-4dbb-4f9d-9398-f59fb630b862


2024-02-08 15:20:20,130 - chromadb.segment.impl.vector.local_persistent_hnsw - WARNING - (local_persistent_hnsw.py:271) - Add of existing embedding ID: cb553733-838a-421b-89bf-c582fe90182a 


Add of existing embedding ID: cb553733-838a-421b-89bf-c582fe90182a


In [25]:
len(simple_ols)

50

In [26]:
object_arrays

[Document(id_='a044bf0f-199b-4cab-822a-b61305ba9495', embedding=None, metadata={'columns': "['x1', 'x2', 'x3', 'y']", 'schema': 'None', 'shape': '(50, 4)'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='9f4c001f1f1ad7d1636d4457b18f1cbb05d3359492cf2267e2666fabf435f140', text='97.98219999874924, 99.84941752810117, 100.9727776594234, 360.87650920565545\n101.00077953260389, 99.87874921228179, 99.35642250227457, 361.50488035486944\n98.5109626677227, 100.7485502397903, 99.46465098250788, 359.8117609861218\n100.77335929310553, 100.03722922045552, 99.86657209922947, 362.2336960397953\n100.97359840386007, 99.1724799721807, 100.16093297144785, 362.1391160315852\n100.18799255929102, 100.55900119891184, 100.61532849440285, 363.29752977180965\n100.9157547652626, 98.61649241995889, 99.06726035297895, 359.7975894964005\n101.04615952660859, 102.00920930524853, 100.16419028246959, 364.8003752715575\n99.46321248760913, 100.23898461781165, 100.4603474082993, 361.

In [23]:
# create a query engine
query_engine = index.as_query_engine()

In [27]:
response = query_engine.query("How many values with in the dataset?")
print_resp(response.response)

2024-02-08 15:12:13,321 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

 The dataset contains a total of 50 rows, as indicated
by the shape of (50, 4) for both sets of data provided.
However, it's important to note that the context
information doesn't explicitly state that there are two
sets of data provided. But based on the alternating
columns in each block of data, we can infer that there
are indeed two separate sets of data with 50 rows each.

************************************************************
 🚩END OF RESPONSE


In [26]:
df.shape

(50, 4)

In [29]:
response = query_engine.query("How many columns' name starts with 'x'?")
print_resp(response.response)

2024-02-08 15:13:24,979 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

 There are three columns' names that start with 'x',
which are 'x1', 'x2', and 'x3'.

************************************************************
 🚩END OF RESPONSE


In [24]:
response = query_engine.query("What is the average of column 'x1'?")
print_resp(response.response)

2024-02-08 15:21:03,508 - httpx - INFO - (_client.py:1027) - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK" 


HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
✅ RESPONSE:
************************************************************

 To find the average of column 'x1', we need to sum all
the values in column 'x1' and then divide by the total
number of rows (50). Here are all the values in column
'x1':  99.07093312420517, 100.84958819151115,
99.72656237514776, ... , 100.97373793175905  Adding all
the values gives us:  99.07093312420517 +
100.84958819151115 + ... + 100.97373793175905 = (sum of
all x1 values)  Now, we need to divide the sum by the
total number of rows, which is 50:  (sum of all x1
values) / 50 = average of column 'x1'  Without
calculating the exact sum, we can see that the average
value lies between 98.73 (lowest value) and 100.97
(highest value). However, without performing the actual
calculation, we cannot provide an exact numerical
answer for the average of column 'x1'.  Please note
that providing the exact average would require further
calculations that go

In [27]:
df.head()

,x1,x2,x3,y
0,97.982200,99.849418,100.972778,360.876509
1,101.000780,99.878749,99.356423,361.504880
2,98.510963,100.748550,99.464651,359.811761
3,100.773359,100.037229,99.866572,362.233696
4,100.973598,99.172480,100.160933,362.139116


In [35]:
print(df['x1'][:5])
print('Mean= ', df['x1'].mean())

0     97.982200
1    101.000780
2     98.510963
3    100.773359
4    100.973598
Name: x1, dtype: float64
Mean=  100.07520939069373
